In [1]:
%run "../Common/Enums"

In [2]:
def CopyDynamicsData(lakeName):
  print("LakeName:", lakeName)
  
  # get last date from the loaded dataset
  LastSyncDate = GetLastSyncDate('accounts', 'dynamics')
  
  print("Reading data since", LastSyncDate)  
  print("Load Dynamics Accounts data")
  PrepareDataBronzeWithoutInstance (
    lakeName = lakeName,
    entity = BronzeTable.Accounts_Dynamics,
    entityName = 'tmpAccounts',
    startDate = LastSyncDate
  )
  
  print("Writing data...")
  
  sourceData = spark.sql("""
    select
      accountid,
      null as accountnumber,
      modifiedon,
      industrycode, 
      current_timestamp() as SilverModifiedUtc
    from tmpAccounts
  """)  
  
  sourceData.registerTempTable("SourceData")
   
  spark.sql("""
    merge into silver.Accounts as t
    using SourceData as s
      on t.accountid = s.accountid
    when matched then update set 
        accountnumber = s.accountnumber
      , modifiedon = s.modifiedon
      , industrycode = s.industrycode
    when not matched then 
      insert ( accountid, AccountNumber, ModifiedOn, IndustryCode, BillingAccountId, Culture, AccountName, Status, IsTest, DataPurgedCompleted, CreationDate 
            , MOdificationDate, OldAccountId, SilverModifiedUtc)
      values (  accountid, AccountNumber, ModifiedOn, IndustryCode, null, null, null, null, null, null, null, null, null, SilverModifiedUtc )
  """);

  print("Writing data completed.")
  
  LastSyncDate = sourceData.agg({"ModifiedOn": "max"}).collect()[0][0]
  print(LastSyncDate)

  if LastSyncDate is not None:
    UpdateLastSyncDate(LastSyncDate, 'accounts', 'dynamics')